# Pre-Processing CSV data for MapAggC
#### General Approach:
1) Read in csv sheet of data.  Export fields into new dataframe.
2) Remove bad rows and elements.
3) Fix date datatype.
4) For ReportingUnit df, include ID value for easy match to shp file in ArcGIS (see below).
8) Export completed df as processed csZv.

#### Files Needed
1) dontopen_RU.csv, 
2) dontopen_AggLJAll.csv

In [1]:
#Needed Libararies
import os
import numpy as np
import pandas as pd
from datetime import datetime
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook
pd.options.display.max_rows  # Increases the length of a printed string from a DataFrame
pd.set_option('display.max_colwidth', -1)

C:\Users\rjame\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


## ReportingUnits Table and Export

#### General Notes
We drop the geometry field at this time.  API returns a encrypted version of it, which we can't work with at this time.  Have to instead match ReportingUnit_dim table to several separate shp files based on Unit Type (e.g., county, HUC8, custom, etc).  Have to build an ID value for this to work.  Each Unit Type file ID and match approach is different, see below.

#### ReportingUnit csv to ArcGIS shp file notes.
Creating ID to link p-csv to shp file works for every ReportUnitType but HUC8, as the ArcGIS shp we use doesn't incldue name in its data.  Have to cheat, and for all HUC8 types, the format should be "2_" + !HUC_8! + "_" + !StateNum!.  For the time, this was easier to do by hand than by code.

Wyoming also includes "... River Planning Basin" in its REportingUnitName.  Easier just to remove by hand.  Changed one WY shp file entry to Snake-Salt, from Salt-Snake.

California custom DAUCO also worked better using ReportingUnitNativeID.

AZ custom AM alsow worked better using ReportingUnitNative ID.  Shp didn't include a StateNum field, I just added it in.

USBR did work okay with ReportingUnitName.  Did have to update the shapefile with StateNum field = 100.

In [2]:
#Working Directory and Input File
workingDir = "C:/Users/rjame/Documents/WSWC Documents/Portal Creation Research"
os.chdir(workingDir)

In [3]:
RU_Input = "dontopen_RU.csv"
df_RU = pd.read_csv(RU_Input)
df_RU

,ReportingUnitID,ReportingUnitUUID,ReportingUnitNativeID,ReportingUnitName,ReportingUnitTypeCV,ReportingUnitUpdateDate,ReportingUnitProductVersion,StateCV,EPSGCodeCV,Geometry
0,20733,NM_C_1,35001,Bernalillo,County,NaN,NaN,NM,EPSG:4326,0xE61000000104110000000684D6C397CC5AC03C6C2233179C41404F90D8EE1EAA5AC00987DEE2E19B41401478279F1EA95AC08AC745B5889A4140EF5696E82CA95AC01E32E54350994140683EE76ED7A85AC00BB43BA4189A414023BDA8DDAFA65AC0A48D23D6E29B41403C4A253CA18F5AC0A51309A69A9B41407C80EECB998F5AC0323D6189077A41409B012EC896895AC048FDF50A0B7A414013807F4A95895AC0227024D0606F41404F07B29E5A9A5AC0A9A3E36A646F41400F9A5DF7569A5AC003965CC5E2714140C005D9B27C9B5AC06B9A779CA27341401AA4E029E4AB5AC0E63BF88903744140B45549641FAE5AC0B2648EE55D6F4140E597C11891C15AC04F07B29E5A6F41400684D6C397CC5AC03C6C2233179C414001000000020000000001000000FFFFFFFF0000000003
1,20734,NM_C_2,35003,Catron,County,NaN,NaN,NM,EPSG:4326,0xE6100000010410000000A2629CBF09435BC046787B1002A04040E8A04B38F4425BC0FEEE1D35264A414092955F0663EE5AC0B262B83A004A414051F701486DEE5AC0D2DF4BE141214140E54526E0D7ED5AC0AED51EF642214140371B2B31CFED5AC00CAF2479AEE94040DE550F9887ED5AC03C33C170AEE94040F705F4C29DED5AC0D8D7BAD408BD40403B527DE717FC5AC0D1E7A38CB8BC4040FB93F8DC09005BC0DC8310902FBD40405A9C31CC09005BC0679C86A8C2994040EBE5779ACC1C5BC00CCC0A45BA994040077AA86DC3225BC01405FA449E9A4040452C62D861295BC0F1D6F9B7CB9A40400773D6ED05435BC0B0898612BD9A4040A2629CBF09435BC046787B1002A0404001000000020000000001000000FFFFFFFF0000000003
2,20735,NM_C_3,35005,Chaves,County,NaN,NaN,NM,EPSG:4326,0xE61000000104200000004A46CEC29E565AC0605B3FFD674D40408D9AAF928F565AC0F4C29D0B237B4040EE3F321D3A545AC0A5BDC117267B40403A78263449545AC0D314014EEF9040402AAA7EA5F3465AC03CC1FEEBDC904040E7C2482F6A405AC05CC64D0D349140401D041DAD6A405AC0F48C7DC9C6914040DD408177F2395AC04CA4349BC791404073B8567BD8395AC098DBBDDC27A7404058AD4CF8A5385AC0ED9DD15625A7404040852348A5385AC08CBB41B456B24040C1374D9F1D395AC06326512FF8B24040E97FB9162D395AC065DEAAEB500B414010E84CDA54325AC0AD84EE92380B41404ED026874F325AC081423D7D04004140A7E7DD5850145AC057EBC4E578FF40403B18B14F000A5AC0B28174B169FF40407DD0B359F5095AC08DEE2076A60A41409A95ED43DEF559C0C11F7EFE7B0A4140151BF33AE2F559C00D71AC8BDBE840408D7F9F71E1ED59C06C96CB46E7E84040CEA8F92AF9ED59C072A774B0FED34040DE7536E49FE059C0A873452921D4404070EF1AF4A5E059C063F20698F9C84040666A12BC21EE59C0546EA296E6C8404008CDAE7B2BEE59C08274B169A5B2404083FB010F0CF159C07C9A931799B240404D86E3F90CF159C0C90050C58D7B40404E469561DC355AC0810A47904A7B4040EFA83121E6355AC04221020EA14240403F1D8F19A8565AC090F98040674240404A46CEC29E565AC0605B3FFD674D404001000000020000000001000000FFFFFFFF0000000003
3,20736,NM_C_4,35006,Cibola,County,NaN,NaN,NM,EPSG:4326,0xE610000001040F000000D8996182F3425BC06858F2C85A59414084FE5E39EF425BC085B24655DB7A414054ABAFAE0A1E5BC0849B8C2AC37A41404209336DFF1D5BC04487C09140A741400DFCA886FDEE5AC0425A63D009A741407898F6CDFDEE5AC00AA2EE0390AC4140B5A9BA4736E85AC0945E9B8D95AC4140810706103EE85AC0FA0B3D62F4A64140DA5548F949CF5AC07BD976DA1AA74140024A438D42C45AC07407B133857A41409DBCC804FCCC5AC05CC823B8917A41401FBB0B9414CD5AC015527E52ED4D41409ACC785BE9CC5AC061AA99B5144A4140E8A04B38F4425BC0FEEE1D35264A4140D8996182F3425BC06858F2C85A59414001000000020000000001000000FFFFFFFF0000000003
4,20737,NM_C_5,35007,Colfax,County,NaN,NaN,NM,EPSG:4326,0xE610000001044D000000C0EC9E3C2C585AC0465F419AB14642402DCE18E604585AC028B9C32632474240677C5F5CAA575AC0001AA54BFF464240199293895B565AC05A1135D1E74742403621AD31E8565AC0A2EC2DE57C4B42400F643DB5FA575AC0CA6DFB1EF54D4240FC17080264575AC044DB3175575042405EF6EB4E77565AC0F641960513514240D8D30E7F4D575AC06C088ECBB8534240D4D347E00F575AC0A777F17EDC544240ECDFF599B3575AC0CE2A33A5F5554240E9297288B8575AC0C6C37B0E2C5742400B7C45B75E575AC0D8614CFA7B574240C09481035A575AC01A6CEA3C2A584240821ABE8575555AC00C74ED0BE85942402101A3CB9B545AC07EA83462665942403BE12538F5535AC041F33977BB5C42405051F52B9D525AC01C28F04E3E5D424093AAED26F8515AC03D49BA66F25B4240EFE6A90EB9515AC058AEB7CD545C424008CA6DFB1E515AC046459C4EB25B4240E8154F3DD2505AC0DAC

In [4]:
#removeing bad rows of df because they are null for some odd reason
df_RU = df_RU[(df_RU.ReportingUnitUUID != 'test')]

#Dropping fields we don't need.
df_RU = df_RU.drop(['Geometry', 'ReportingUnitUpdateDate', 'ReportingUnitProductVersion'], axis=1)

#Fixing Index
df_RU = df_RU.reset_index()
df_RU = df_RU.drop(columns=['index'])

df_RU

,ReportingUnitID,ReportingUnitUUID,ReportingUnitNativeID,ReportingUnitName,ReportingUnitTypeCV,StateCV,EPSGCodeCV
0,20733,NM_C_1,35001,Bernalillo,County,NM,EPSG:4326
1,20734,NM_C_2,35003,Catron,County,NM,EPSG:4326
2,20735,NM_C_3,35005,Chaves,County,NM,EPSG:4326
3,20736,NM_C_4,35006,Cibola,County,NM,EPSG:4326
4,20737,NM_C_5,35007,Colfax,County,NM,EPSG:4326
...,...,...,...,...,...,...,...
1182,24907,CAag_RU547,DAU40323,Upper Russian,Detailed Analysis Units by County,CA,EPSG:4326
1183,24908,CAag_RU548,DAU40423,Middle Russian,Detailed Analysis Units by County,CA,EPSG:4326
1184,24909,CAag_RU549,DAU40449,Middle Russian,Detailed Analysis Units by County,CA,EPSG:4326
1185,24910,CAag_RU550,DAU40523,Dry Creek,Detailed Analysis Units by County,CA,EPSG:4326


In [5]:
# #Creating Linking Element with Shape File.
# #Format Ex: "1_" + !NAME! + "_" + !StateNum!

# stateNumDic = {
#     "UT" : "46",
#     "NM" : "35",
#     "NE" : "41",
#     "CO" : "41",
#     "WY" : "47",
#     "CA" : "49",
#     "AZ" : "48",
#     "TX" : "37",
#     "US" : "100"
# }

# def createTypeNameNum(colrowValueA, colrowValueB, colrowValueC):
#     StringA = colrowValueA.strip()  # ReportingUnitTypeCV
#     StringB = colrowValueB.strip()  # ReportingUnitName
#     StringC = colrowValueC.strip()  # state for StateNumDic
    
#     outStringA = ""
    
#     if StringA == "County":
#         outStringA = "1"
#     if StringA == "HUC8":
#         outStringA = "2"
#     if StringA == "Basin":
#         outStringA = "3"
#     if StringA == "Planning Area":
#         outStringA = "4"
#     if StringA == "Active Management Area":
#         outStringA = "5"
#     if StringA == "Tributary":
#         outStringA = "6"
#     if StringA == "Subarea":
#         outStringA = "7"
#     if StringA == "Hydrologic Region":
#         outStringA = "8"
    
#     outList = outStringA + "_" + StringB + "_" + stateNumDic[StringC]
#     return outList

# df_RU['TypeNameNum'] = df_RU.apply(lambda row: createTypeNameNum(row['ReportingUnitTypeCV'], row['ReportingUnitName'], row['StateCV']), axis=1)
# df_RU

In [6]:
# #Creating Linking Element with Shape File.
# #Format Ex: "1_" + !ID! + "_" + !StateNum!

# stateNumDic = {
#     "UT" : "46",
#     "NM" : "35",
#     "NE" : "41",
#     "CO" : "42",
#     "WY" : "47",
#     "CA" : "49",
#     "AZ" : "48",
#     "TX" : "37",
#     "US" : "100"
# }


# def createTypeNameNum(colrowValueA, colrowValueB, colrowValueC):
#     StringA = colrowValueA.strip()  # ReportingUnitTypeCV
#     StringB = colrowValueB.strip()  # ReportingUnitNativeID
#     StringC = colrowValueC.strip()  # state for StateNumDic
    
#     outStringA = ""
    
#     if StringA == "County":
#         outStringA = "1"
#     if StringA == "HUC8":
#         outStringA = "2"
#     if StringA == "Basin":
#         outStringA = "3"
#     if StringA == "Planning Area":
#         outStringA = "4"
#     if StringA == "Active Management Area":
#         outStringA = "5"
#     if StringA == "Tributary":
#         outStringA = "6"
#     if StringA == "Subarea":
#         outStringA = "7"
#     if StringA == "Hydrologic Region":
#         outStringA = "8"
    
#     outList = outStringA + "_" + StringB + "_" + stateNumDic[StringC]
#     return outList

# df_RU['TypeIDNum'] = df_RU.apply(lambda row: createTypeNameNum(row['ReportingUnitTypeCV'], row['ReportingUnitNativeID'], row['StateCV']), axis=1)
# df_RU

In [7]:
#The Ouput
df_RU.to_csv('Pagg_ReportingUnit.csv', index=False)

## AggregatedAmounts_withOrg

In [8]:
AAwO_Input = "dontopen_AggLJAll.csv"
df_AAwO = pd.read_csv(AAwO_Input)
df_AAwO.head(5)

C:\Users\rjame\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,AggregatedAmountID,OrganizationID,ReportingUnitID,VariableSpecificID,WaterSourceID,MethodID,ReportYearCV,Amount,TimeframeStartID,TimeframeEndID,ReportingUnitUUID,ReportingUnitNativeID,ReportingUnitName,ReportingUnitTypeCV,TimeframeStartID.1,TimeframeEndID.1,WaterSourceTypeCV,VariableCV,VariableSpecificCV,ApplicableResourceTypeCV,MethodTypeCV,State,BeneficialUseCV
0,532165,7,20733,116,559476,78,2015,0.00,37752.0,36160.0,NM_C_1,35001,Bernalillo,County,37752.0,36160.0,Surface Water,Withdrawal,Withdrawal,Surface Ground,Water Use,NM,NaN
1,532166,7,20733,116,559476,78,2015,45152.23,37752.0,36160.0,NM_C_1,35001,Bernalillo,County,37752.0,36160.0,Surface Water,Withdrawal,Withdrawal,Surface Ground,Water Use,NM,NaN
2,532167,7,20733,116,559476,78,2015,0.00,37752.0,36160.0,NM_C_1,35001,Bernalillo,County,37752.0,36160.0,Surface Water,Withdrawal,Withdrawal,Surface Ground,Water Use,NM,NaN
3,532168,7,20733,116,559476,78,2015,0.00,37752.0,36160.0,NM_C_1,35001,Bernalillo,County,37752.0,36160.0,Surface Water,Withdrawal,Withdrawal,Surface Ground,Water Use,NM,NaN
4,532169,7,20733,116,559476,78,2015,0.00,37752.0,36160.0,NM_C_1,35001,Bernalillo,County,37752.0,36160.0,Surface Water,Withdrawal,Withdrawal,Surface Ground,Water Use,NM,NaN


In [9]:
date_Input = "dontopen_Date_dim.csv"
df_date = pd.read_csv(date_Input)
df_date.head(5)

,DateID,Date,Year
0,1,2013-10-01,2013
1,2,2003-09-01,2003
2,3,2003-10-01,2003
3,4,2006-08-01,2006
4,5,2006-09-01,2006


In [10]:
#Retreiving TimeframeStart from Dates_dim Table
TimeframeStartdict = pd.Series(df_date.Date.values, index = df_date.DateID).to_dict()

# For creating TimeframeStart
def retrieveTimeframeStart(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        outList = ''
    else:
        val1 = colrowValue
        try:
            outList = TimeframeStartdict[val1]
        except:
            outList = colrowValue
    return outList

df_AAwO['TimeframeStart'] = df_AAwO.apply(lambda row: retrieveTimeframeStart(row['TimeframeStartID']), axis=1)
df_AAwO.head(5)

,AggregatedAmountID,OrganizationID,ReportingUnitID,VariableSpecificID,WaterSourceID,MethodID,ReportYearCV,Amount,TimeframeStartID,TimeframeEndID,ReportingUnitUUID,ReportingUnitNativeID,ReportingUnitName,ReportingUnitTypeCV,TimeframeStartID.1,TimeframeEndID.1,WaterSourceTypeCV,VariableCV,VariableSpecificCV,ApplicableResourceTypeCV,MethodTypeCV,State,BeneficialUseCV,TimeframeStart
0,532165,7,20733,116,559476,78,2015,0.00,37752.0,36160.0,NM_C_1,35001,Bernalillo,County,37752.0,36160.0,Surface Water,Withdrawal,Withdrawal,Surface Ground,Water Use,NM,NaN,2020-01-01
1,532166,7,20733,116,559476,78,2015,45152.23,37752.0,36160.0,NM_C_1,35001,Bernalillo,County,37752.0,36160.0,Surface Water,Withdrawal,Withdrawal,Surface Ground,Water Use,NM,NaN,2020-01-01
2,532167,7,20733,116,559476,78,2015,0.00,37752.0,36160.0,NM_C_1,35001,Bernalillo,County,37752.0,36160.0,Surface Water,Withdrawal,Withdrawal,Surface Ground,Water Use,NM,NaN,2020-01-01
3,532168,7,20733,116,559476,78,2015,0.00,37752.0,36160.0,NM_C_1,35001,Bernalillo,County,37752.0,36160.0,Surface Water,Withdrawal,Withdrawal,Surface Ground,Water Use,NM,NaN,2020-01-01
4,532169,7,20733,116,559476,78,2015,0.00,37752.0,36160.0,NM_C_1,35001,Bernalillo,County,37752.0,36160.0,Surface Water,Withdrawal,Withdrawal,Surface Ground,Water Use,NM,NaN,2020-01-01


In [11]:
#Retreiving TimeframeEnd from Dates_dim Table
TimeframeEnddict = pd.Series(df_date.Date.values, index = df_date.DateID).to_dict()

# For creating TimeframeEnd
def retrieveTimeframeEnd(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        outList = ''
    else:
        val1 = colrowValue
        try:
            outList = TimeframeEnddict[val1]
        except:
            outList = colrowValue
    return outList

df_AAwO['TimeframeEnd'] = df_AAwO.apply(lambda row: retrieveTimeframeEnd(row['TimeframeEndID']), axis=1)
df_AAwO.head(5)

,AggregatedAmountID,OrganizationID,ReportingUnitID,VariableSpecificID,WaterSourceID,MethodID,ReportYearCV,Amount,TimeframeStartID,TimeframeEndID,ReportingUnitUUID,ReportingUnitNativeID,ReportingUnitName,ReportingUnitTypeCV,TimeframeStartID.1,TimeframeEndID.1,WaterSourceTypeCV,VariableCV,VariableSpecificCV,ApplicableResourceTypeCV,MethodTypeCV,State,BeneficialUseCV,TimeframeStart,TimeframeEnd
0,532165,7,20733,116,559476,78,2015,0.00,37752.0,36160.0,NM_C_1,35001,Bernalillo,County,37752.0,36160.0,Surface Water,Withdrawal,Withdrawal,Surface Ground,Water Use,NM,NaN,2020-01-01,2020-12-31
1,532166,7,20733,116,559476,78,2015,45152.23,37752.0,36160.0,NM_C_1,35001,Bernalillo,County,37752.0,36160.0,Surface Water,Withdrawal,Withdrawal,Surface Ground,Water Use,NM,NaN,2020-01-01,2020-12-31
2,532167,7,20733,116,559476,78,2015,0.00,37752.0,36160.0,NM_C_1,35001,Bernalillo,County,37752.0,36160.0,Surface Water,Withdrawal,Withdrawal,Surface Ground,Water Use,NM,NaN,2020-01-01,2020-12-31
3,532168,7,20733,116,559476,78,2015,0.00,37752.0,36160.0,NM_C_1,35001,Bernalillo,County,37752.0,36160.0,Surface Water,Withdrawal,Withdrawal,Surface Ground,Water Use,NM,NaN,2020-01-01,2020-12-31
4,532169,7,20733,116,559476,78,2015,0.00,37752.0,36160.0,NM_C_1,35001,Bernalillo,County,37752.0,36160.0,Surface Water,Withdrawal,Withdrawal,Surface Ground,Water Use,NM,NaN,2020-01-01,2020-12-31


In [12]:
#Dropping fields we don't need.
df_AAwO = df_AAwO.drop(['TimeframeStartID', 'TimeframeEndID'], axis=1)
df_AAwO.head(5)

,AggregatedAmountID,OrganizationID,ReportingUnitID,VariableSpecificID,WaterSourceID,MethodID,ReportYearCV,Amount,ReportingUnitUUID,ReportingUnitNativeID,ReportingUnitName,ReportingUnitTypeCV,TimeframeStartID.1,TimeframeEndID.1,WaterSourceTypeCV,VariableCV,VariableSpecificCV,ApplicableResourceTypeCV,MethodTypeCV,State,BeneficialUseCV,TimeframeStart,TimeframeEnd
0,532165,7,20733,116,559476,78,2015,0.00,NM_C_1,35001,Bernalillo,County,37752.0,36160.0,Surface Water,Withdrawal,Withdrawal,Surface Ground,Water Use,NM,NaN,2020-01-01,2020-12-31
1,532166,7,20733,116,559476,78,2015,45152.23,NM_C_1,35001,Bernalillo,County,37752.0,36160.0,Surface Water,Withdrawal,Withdrawal,Surface Ground,Water Use,NM,NaN,2020-01-01,2020-12-31
2,532167,7,20733,116,559476,78,2015,0.00,NM_C_1,35001,Bernalillo,County,37752.0,36160.0,Surface Water,Withdrawal,Withdrawal,Surface Ground,Water Use,NM,NaN,2020-01-01,2020-12-31
3,532168,7,20733,116,559476,78,2015,0.00,NM_C_1,35001,Bernalillo,County,37752.0,36160.0,Surface Water,Withdrawal,Withdrawal,Surface Ground,Water Use,NM,NaN,2020-01-01,2020-12-31
4,532169,7,20733,116,559476,78,2015,0.00,NM_C_1,35001,Bernalillo,County,37752.0,36160.0,Surface Water,Withdrawal,Withdrawal,Surface Ground,Water Use,NM,NaN,2020-01-01,2020-12-31


In [13]:
df_AAwO['BeneficialUseCV'].fillna('Unspecified', inplace=True)
df_AAwO

,AggregatedAmountID,OrganizationID,ReportingUnitID,VariableSpecificID,WaterSourceID,MethodID,ReportYearCV,Amount,ReportingUnitUUID,ReportingUnitNativeID,ReportingUnitName,ReportingUnitTypeCV,TimeframeStartID.1,TimeframeEndID.1,WaterSourceTypeCV,VariableCV,VariableSpecificCV,ApplicableResourceTypeCV,MethodTypeCV,State,BeneficialUseCV,TimeframeStart,TimeframeEnd
0,532165,7,20733,116,559476,78,2015,0.00,NM_C_1,35001,Bernalillo,County,37752.0,36160.0,Surface Water,Withdrawal,Withdrawal,Surface Ground,Water Use,NM,Unspecified,2020-01-01,2020-12-31
1,532166,7,20733,116,559476,78,2015,45152.23,NM_C_1,35001,Bernalillo,County,37752.0,36160.0,Surface Water,Withdrawal,Withdrawal,Surface Ground,Water Use,NM,Unspecified,2020-01-01,2020-12-31
2,532167,7,20733,116,559476,78,2015,0.00,NM_C_1,35001,Bernalillo,County,37752.0,36160.0,Surface Water,Withdrawal,Withdrawal,Surface Ground,Water Use,NM,Unspecified,2020-01-01,2020-12-31
3,532168,7,20733,116,559476,78,2015,0.00,NM_C_1,35001,Bernalillo,County,37752.0,36160.0,Surface Water,Withdrawal,Withdrawal,Surface Ground,Water Use,NM,Unspecified,2020-01-01,2020-12-31
4,532169,7,20733,116,559476,78,2015,0.00,NM_C_1,35001,Bernalillo,County,37752.0,36160.0,Surface Water,Withdrawal,Withdrawal,Surface Ground,Water Use,NM,Unspecified,2020-01-01,2020-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117782,664345,13,24911,108,564502,86,2012,0.00,CAag_RU551,DAU40549,Dry Creek,Detailed Analysis Units by County,NaN,NaN,"Groundwater, Surface Water",Consumptive Use,Consumptive Use,Unspecified,Water Use,CA,Unspecified,,
117783,664346,13,24911,108,564502,86,2012,0.00,CAag_RU551,DAU40549,Dry Creek,Detailed Analysis Units by County,NaN,NaN,"Groundwater, Surface Water",Consumptive Use,Consumptive Use,Unspecified,Water Use,CA,Unspecified,,
117784,664347,13,24911,108,564502,86,2012,17100.00,CAag_RU551,DAU40549,Dry Creek,Detailed Analysis Units by County,NaN,NaN,"Groundwater, Surface Water",Consumptive Use,Consumptive Use,Unspecified,Water Use,CA,Unspecified,,
117785,664348,13,24911,108,564502,86,2012,49900.00,CAag_RU551,DAU40549,Dry Creek,Detailed Analysis Units by County,NaN,NaN,"Groundwater, Surface Water",Consumptive Use,Consumptive Use,Unspecified,Water Use,CA,Unspecified,,


In [14]:
#The Ouput
df_AAwO.to_csv('Pagg_AggregatedAmountsAll.csv', index=False)